# OpenAI Assitants API

The [Assistants API](https://platform.openai.com/docs/assistants/overview) allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, Retrieval, and Function calling.

TruLens can be easily integrated with the assistants API to provide the same observability tooling you are used to when building with other frameworks.


[**Important**] Notice in this example notebook, we are using Assistants API V1 (hence the pinned version of `openai` below) so that we can evaluate against retrieved source.
At some very recent point in time as of April 2024, OpenAI removed the ["quote" attribute from file citation object in Assistants API V2](https://platform.openai.com/docs/api-reference/messages/object#messages/object-content) due to stability issue of this feature. See response from OpenAI staff https://community.openai.com/t/assistant-api-always-return-empty-annotations/489285/48
 

In [1]:
#!pip install trulens-eval openai==1.14.3 # pinned openai version to avoid breaking changes 

## Set keys

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

## Create the assistant

Let's create a new assistant that answers questions about the famous *Paul Graham Essay*.

The easiest way to get it is to download it via this link and save it in a folder called data. You can do so with the following command

In [3]:
!wget https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt -P data/

--2024-04-25 18:07:33--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham_essay.txt.2’

paul_graham_essay.t 100%[===================>]  73.28K  --.-KB/s    in 0.007s  

2024-04-25 18:07:33 (9.58 MB/s) - ‘data/paul_graham_essay.txt.2’ saved [75042/75042]



## Add TruLens

In [4]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


## Create a thread (V1 Assistants)

In [5]:
from openai import OpenAI

class RAG_with_OpenAI_Assistant:
    def __init__(self):
        client = OpenAI()
        self.client = client

        # upload the file\
        file = client.files.create(
        file=open("data/paul_graham_essay.txt", "rb"),
        purpose='assistants'
        )

        # create the assistant with access to a retrieval tool
        assistant = client.beta.assistants.create(
            name="Paul Graham Essay Assistant",
            instructions="You are an assistant that answers questions about Paul Graham.",
            tools=[{"type": "retrieval"}],
            model="gpt-4-turbo-preview",
            file_ids=[file.id]
        )
        
        self.assistant = assistant

    @instrument
    def retrieve_and_generate(self, query: str) -> str:
        """
        Retrieve relevant text by creating and running a thread with the OpenAI assistant.
        """
        self.thread = self.client.beta.threads.create()
        self.message = self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=query
        )

        run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
            instructions="Please answer any questions about Paul Graham."
        )

        # Wait for the run to complete
        import time
        while run.status in ['queued', 'in_progress', 'cancelling']:
            time.sleep(1)
            run = self.client.beta.threads.runs.retrieve(
                thread_id=self.thread.id,
                run_id=run.id
            )

        if run.status == 'completed':
            messages = self.client.beta.threads.messages.list(
                thread_id=self.thread.id
            )
            response = messages.data[0].content[0].text.value
            quote = messages.data[0].content[0].text.annotations[0].file_citation.quote
        else:
            response = "Unable to retrieve information at this time."

        return response, quote
    
rag = RAG_with_OpenAI_Assistant()

## Create feedback functions

In [6]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

import numpy as np

provider = OpenAI()

grounded = Groundedness(groundedness_provider=provider)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve_and_generate.rets[1])
    .on(Select.RecordCalls.retrieve_and_generate.rets[0])
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve_and_generate.args.query)
    .on(Select.RecordCalls.retrieve_and_generate.rets[0])
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve_and_generate.args.query)
    .on(Select.RecordCalls.retrieve_and_generate.rets[1])
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve_and_generate.rets[1] .
✅ In Groundedness, input statement will be set to __record__.app.retrieve_and_generate.rets[0] .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve_and_generate.args.query .
✅ In Answer Relevance, input response will be set to __record__.app.retrieve_and_generate.rets[0] .
✅ In Context Relevance, input question will be set to __record__.app.retrieve_and_generate.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve_and_generate.rets[1] .


[nltk_data] Downloading package punkt to /home/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'OpenAI Assistant RAG',
    feedbacks = [f_groundedness, f_answer_relevance, f_context_relevance])

In [8]:
with tru_rag:
    rag.retrieve_and_generate("How did paul graham grow up?")

In [10]:
from trulens_eval import Tru

tru.get_leaderboard()

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
OpenAI Assistant RAG,0.307692,1.0,0.4,38.0,0.0


In [11]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…